In [1]:
import os
import pandas as pd
import numpy as np
import math
from sklearn.cluster import KMeans
from sklearn.metrics import normalized_mutual_info_score
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn

from utils import *
from autoencoders.basic_autoencoder import BasicAutoencoder
from autoencoders.all_to_all_autoencoder import AllToAllAutoencoder
from autoencoders.all_to_cat_autoencoder import AllToCatAutoencoder
from autoencoders.cat_to_cat_autoencoder import CatToCatAutoencoder
from load_datasets import load_all_datasets

In [2]:
device = "cpu"
os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
torch.manual_seed(0)
torch.use_deterministic_algorithms(True)
np.random.seed(0)

datasets = load_all_datasets(max_rows=5000)
accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}
for d in datasets:
    print(f"{d.name}: Input dim: {d.input_dim}; Cat dim: {d.cat_dim}; Cont dim: {d.cont_dim}")

Abalone: Input dim: 9; Cat dim: 2; Cont dim: 7
Auction Verification: Input dim: 13; Cat dim: 12; Cont dim: 1
Bank Marketing: Input dim: 22; Cat dim: 17; Cont dim: 5
Breast Cancer: Input dim: 45; Cat dim: 45; Cont dim: 0
Census Income: Input dim: 58; Cat dim: 52; Cont dim: 6
Credit Approval: Input dim: 28; Cat dim: 22; Cont dim: 6
Heart Disease: Input dim: 17; Cat dim: 11; Cont dim: 6
Soybean Disease: Input dim: 55; Cat dim: 55; Cont dim: 0


In [3]:
datasets = load_all_datasets(max_rows=5000)

accuracies = {d.name: {} for d in datasets}
nmis = {d.name: {} for d in datasets}

for d in datasets:
    print(f"{d.name}: Input dim: {d.input_dim}; Cat dim: {d.cat_dim}; Cont dim: {d.cont_dim}")

Abalone: Input dim: 9; Cat dim: 2; Cont dim: 7
Auction Verification: Input dim: 13; Cat dim: 12; Cont dim: 1
Bank Marketing: Input dim: 22; Cat dim: 17; Cont dim: 5
Breast Cancer: Input dim: 45; Cat dim: 45; Cont dim: 0
Census Income: Input dim: 58; Cat dim: 52; Cont dim: 6
Credit Approval: Input dim: 28; Cat dim: 22; Cont dim: 6
Heart Disease: Input dim: 17; Cat dim: 11; Cont dim: 6
Soybean Disease: Input dim: 55; Cat dim: 55; Cont dim: 0


In [4]:
# Basic Autoencoder with no Categorial Embeddings
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = len(d.cat_cols) + len(d.cont_cols)
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)

    ae = BasicAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = torch.tensor(d.df[d.cont_cols].values, dtype=torch.float).detach().to(device)
    features = ae.encode(cat, cont).detach().cpu().numpy()
    kmeans = KMeans(n_clusters=d.n_targets, init="random", max_iter=300, n_init=1, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["No Embeddings AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["No Embeddings AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=8, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6, out_features=4, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=4, out_features=2, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=4, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=4, out_features=6, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=6, out_features=8, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.157973
Epoch 50/100 - Batch Reconstruction loss: 0.157587
Epoch 75/100 - Batch Reconstruction loss: 0.157757
Epoch 100/100 - Batch Reconstruction loss: 0.156816
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=7, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6, o

In [5]:
# All Columns to All Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = sum(d for _, d in d.embedding_sizes) + len(d.cont_cols)
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))))
    print(encoder, decoder)

    ae = AllToAllAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = torch.tensor(d.df[d.cont_cols].values, dtype=torch.float).detach().to(device)
    features = ae.encode(cat, cont).detach().cpu().numpy()
    kmeans = KMeans(n_clusters=d.n_targets, init="random", max_iter=300, n_init=1, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["All cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["All cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=9, out_features=7, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=7, out_features=5, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=5, out_features=2, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=2, out_features=5, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=5, out_features=7, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=7, out_features=9, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 0.357466
Epoch 50/100 - Batch Reconstruction loss: 0.242714
Epoch 75/100 - Batch Reconstruction loss: 0.241585
Epoch 100/100 - Batch Reconstruction loss: 0.238602
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11

In [6]:
# Categorial Columns to Categorial Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = sum(d for _, d in d.embedding_sizes)
    encoder, decoder = build_autoencoder(input_dim, input_dim, max(1, round(math.log2(input_dim))), output_fn=nn.Sigmoid)
    print(encoder, decoder)

    ae = CatToCatAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = d.df[d.cont_cols].values
    scaled_cat_features = StandardScaler().fit_transform(ae.encode(cat, cont).detach().cpu().numpy())
    features = np.concatenate((scaled_cat_features, cont), axis=1)
    kmeans = KMeans(n_clusters=d.n_targets, init="random", max_iter=300, n_init=1, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["Cat Cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["Cat Cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=2, out_features=1, bias=True)
  (1): Sigmoid()
) Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): Sigmoid()
)
Epoch 25/100 - Batch Reconstruction loss: 5.550064
Epoch 50/100 - Batch Reconstruction loss: 6.126270
Epoch 75/100 - Batch Reconstruction loss: 6.506631
Epoch 100/100 - Batch Reconstruction loss: 6.809322
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=12, out_features=10, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=10, out_features=8, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=8, out_features=6, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=6, out_features=3, bias=True)
  (7): Sigmoid()
) Sequential(
  (0): Linear(in_features=3, out_features=6, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=6, out_features=8, bias=True)
  (3): LeakyReLU(neg

In [7]:
# All Columns to Categorial Columns Autoencoder
for d in datasets:
    print(f"Calculating for {d.name}...")
    input_dim = sum(d for _, d in d.embedding_sizes) + len(d.cont_cols)
    encoder, decoder = build_autoencoder(input_dim, sum(d for _, d in d.embedding_sizes), max(1, round(math.log2(sum(d for _, d in d.embedding_sizes)))))
    print(encoder, decoder)

    ae = AllToCatAutoencoder(encoder, decoder, d.embedding_sizes, device=device)
    ae.fit(d.dataloader, n_epochs=100, lr=0.001)

    cat = torch.tensor(d.df[d.cat_cols].values, dtype=torch.int).detach().to(device)
    cont = torch.tensor(d.df[d.cont_cols].values, dtype=torch.float).detach().to(device)
    features = np.concatenate((ae.encode(cat, cont).detach().cpu().numpy(), cont.cpu().numpy()), axis=1)
    kmeans = KMeans(n_clusters=d.n_targets, init="random", max_iter=300, n_init=1, random_state=0, algorithm="lloyd").fit(features)

    nmis[d.name]["All cols to cat cols AE"] = normalized_mutual_info_score(d.y, kmeans.labels_)
    accuracies[d.name]["All cols to cat cols AE"] = cluster_accuracy(d.y, kmeans.labels_)

Calculating for Abalone...
Sequential(
  (0): Linear(in_features=9, out_features=1, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear(in_features=1, out_features=2, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
)
Epoch 25/100 - Batch Reconstruction loss: 2.709342
Epoch 50/100 - Batch Reconstruction loss: 1.027222
Epoch 75/100 - Batch Reconstruction loss: 0.470571
Epoch 100/100 - Batch Reconstruction loss: 0.462656
Calculating for Auction Verification...
Sequential(
  (0): Linear(in_features=13, out_features=11, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Linear(in_features=11, out_features=9, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=9, out_features=7, bias=True)
  (5): LeakyReLU(negative_slope=0.01)
  (6): Linear(in_features=7, out_features=5, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): LeakyReLU(negative_slope=0.01)
) Sequential(
  (0): Linear

In [8]:
pd.DataFrame(nmis.values(), index=nmis.keys()).round(4)

,No Embeddings AE,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.1559,0.1633,0.1745,0.1694
Auction Verification,0.0003,0.0002,0.0029,0.0035
Bank Marketing,0.0006,0.0000,0.0255,0.0172
Breast Cancer,0.7602,0.6493,0.7180,0.6608
Census Income,0.0000,0.1167,0.0041,0.1065
Credit Approval,0.0089,0.0961,0.0020,0.0295
Heart Disease,0.1760,0.1503,0.1297,0.0983
Soybean Disease,0.4765,0.3026,0.5404,0.3862


In [9]:
pd.DataFrame(accuracies.values(), index=accuracies.keys()).round(4)

,No Embeddings AE,All cols AE,Cat Cols AE,All cols to cat cols AE
Abalone,0.1154,0.1379,0.1417,0.1405
Auction Verification,0.5507,0.6197,0.6931,0.5766
Bank Marketing,0.6022,0.7430,0.7930,0.5714
Breast Cancer,0.9634,0.9356,0.9546,0.9283
Census Income,0.7198,0.7322,0.5520,0.5160
Credit Approval,0.5605,0.6539,0.5237,0.6034
Heart Disease,0.3311,0.3378,0.3512,0.3010
Soybean Disease,0.3488,0.2384,0.4431,0.3737
